# Alles importen

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
import pickle


# Secundaire data inladen:

In [10]:
df = pd.read_csv('Synthetic_Fraud_Dataset_md2.csv')
df.sample(15)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
5415,566,PAYMENT,114581.187813,C1606266817,112356.448337,0.000000,M3356719937,48269.231454,162850.419267,0
10768,74,TRANSFER,15627.067893,C3708862100,108765.863969,93138.796076,M5942036455,106551.785488,122178.853381,0
43080,696,TRANSFER,92308.132056,C9951705170,18468.758004,0.000000,M4283389505,109225.951544,201534.083599,0
24954,537,PAYMENT,118744.789585,C7581319109,104034.609152,0.000000,M2656803174,233969.907772,352714.697357,0
46762,412,CASH_OUT,27486.599447,C9984646972,61249.038685,33762.439238,M7329682617,207273.375533,234759.974980,0
15130,244,PAYMENT,39144.298983,C4785839726,104785.759875,65641.460893,M1671005024,156684.737492,195829.036474,0
37337,709,PAYMENT,119261.792688,C1194781497,113991.261507,0.000000,M3026121174,140898.558221,260160.350909,0
49890,617,TRANSFER,82266.241873,C7641611512,66050.124559,0.000000,M4096557947,127617.839319,209884.081192,0
31488,639,TRANSFER,2297.852580,C6410267398,42264.628158,39966.775578,M8644848027,156154.327087,158452.179667,0
22702,42,PAYMENT,43410.359844,C7167670280,153224.056177,109813.696333,M5963323743,89870.178366,133280.538210,0


# Onnodige data droppen en feature engineering toepassen:
## Feature engineering: bereken balansafwijkingen voor verzender en ontvanger. Deze waarden zouden nul moeten zijn als het saldo correct wordt bijgewerkt na de transactie. Niet-nul waarden kunnen wijzen op frauduleuze transacties.

In [11]:
# droppen onnodige features
df = df.drop(['step', 'nameOrig', 'nameDest'], axis=1)

# feature engineering
df['errorBalanceOrig'] = df.newbalanceOrig + df.amount - df.oldbalanceOrg
df['errorBalanceDest'] = df.oldbalanceDest + df.amount - df.newbalanceDest

# encode categorical labels
encoder = LabelEncoder()
df["type"] = encoder.fit_transform(df["type"])

# define x en y
X = df.drop(columns=["isFraud"])  # Features
y = df["isFraud"]  # Doelvariabele

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Opgeslagen beste XGBoost model laden en toepassen op een 5-foudige cross validation en scores opslaan

In [7]:
import pickle
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Load the saved model & threshold
with open("fraud_detection_model_mandy_nieuw.pkl", "rb") as file:
    loaded_data = pickle.load(file)

loaded_model = loaded_data["model"]
loaded_threshold = loaded_data["threshold"]

# Force the model to use CPU (disable GPU)
loaded_model.set_params(tree_method='hist')  # Ensure XGBoost runs on CPU DEZE ZORGT ER DUS VOOR DAT T OP CPU GAAT IPV GPU

print(f"Loaded Model with Saved Threshold: {loaded_threshold:.4f}")

# Perform 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store scores
precision_scores, recall_scores, f1_scores, roc_auc_scores = [], [], [], []

for train_idx, test_idx in cv.split(X_train, y_train):
    X_fold_train, X_fold_test = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_fold_train, y_fold_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

    # Train on the current fold
    loaded_model.fit(X_fold_train, y_fold_train)

    # Predict probabilities
    y_fold_proba = loaded_model.predict_proba(X_fold_test)[:, 1]

    # Apply the saved threshold
    y_fold_pred = (y_fold_proba >= loaded_threshold).astype(int)

    # Calculate metrics
    precision_scores.append(precision_score(y_fold_test, y_fold_pred))
    recall_scores.append(recall_score(y_fold_test, y_fold_pred))
    f1_scores.append(f1_score(y_fold_test, y_fold_pred))
    roc_auc_scores.append(roc_auc_score(y_fold_test, y_fold_proba))  # ROC-AUC uses probabilities

# Print mean and standard deviation of scores
print(f"Precision: Mean = {np.mean(precision_scores):.4f}, Std = {np.std(precision_scores):.4f}")
print(f"Recall: Mean = {np.mean(recall_scores):.4f}, Std = {np.std(recall_scores):.4f}")
print(f"F1 Score: Mean = {np.mean(f1_scores):.4f}, Std = {np.std(f1_scores):.4f}")
print(f"ROC-AUC: Mean = {np.mean(roc_auc_scores):.4f}, Std = {np.std(roc_auc_scores):.4f}")


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:00] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:363: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:00] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:388: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:00] WARNING: /Users/runn

Loaded Model with Saved Threshold: 0.9974


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [15:19:01] WARNING: /Users/runner/work/xgboost/xgboost/src

Precision: Mean = 0.9181, Std = 0.0113
Recall: Mean = 0.8981, Std = 0.0235
F1 Score: Mean = 0.9079, Std = 0.0145
ROC-AUC: Mean = 0.9583, Std = 0.0088
